In [3]:
!pip install dash

import pandas as pd
import plotly.graph_objs as go
import dash
from dash import html, dcc
import numpy as np

# Função para coletar dados da API do Banco Central
def carregar_dados_bcb(codigo_serie, data_inicio, data_fim):
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_serie}/dados?formato=csv&dataInicial={data_inicio}&dataFinal={data_fim}"
    df = pd.read_csv(url, sep=';')
    df['data'] = pd.to_datetime(df['data'], dayfirst=True)
    df['valor'] = pd.to_numeric(df['valor'].str.replace(',', '.'), errors='coerce')
    return df.rename(columns={'data': 'Data', 'valor': 'Valor'})

# Período personalizável
inicio = "01/01/2021"
fim = "31/03/2024"

# Códigos do SGS
dados = {
    'Selic': carregar_dados_bcb(4189, inicio, fim),
    'IPCA': carregar_dados_bcb(433, inicio, fim),
    'USD/BRL': carregar_dados_bcb(1, inicio, fim)
}

# Unificar as séries em um único dataframe
df_merged = dados['Selic'].copy()
df_merged.rename(columns={'Valor': 'Selic'}, inplace=True)

for nome, df in dados.items():
    if nome != 'Selic':
        df_merged = pd.merge(df_merged, df[['Data', 'Valor']].rename(columns={'Valor': nome}), on='Data', how='outer')

df_merged.sort_values('Data', inplace=True)
df_merged.reset_index(drop=True, inplace=True)

# App Dash
app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.H1("Análise de Correlação: Selic, IPCA e USD/BRL"),

    dcc.Graph(
        figure={
            'data': [
                go.Scatter(x=df_merged['Data'], y=df_merged['Selic'], name='Selic'),
                go.Scatter(x=df_merged['Data'], y=df_merged['IPCA'], name='IPCA'),
                go.Scatter(x=df_merged['Data'], y=df_merged['USD/BRL'], name='USD/BRL')
            ],
            'layout': go.Layout(title='Evolução Temporal das Séries', xaxis_title='Data', yaxis_title='Valor')
        }
    ),

    html.H2("Mapa de Correlação"),
    dcc.Graph(
        figure=go.Figure(
            data=go.Heatmap(
                z=df_merged[['Selic', 'IPCA', 'USD/BRL']].corr().values,
                x=['Selic', 'IPCA', 'USD/BRL'],
                y=['Selic', 'IPCA', 'USD/BRL'],
                colorscale='Viridis',
                zmin=-1, zmax=1
            )
        ).update_layout(title='Correlação entre Variáveis Econômicas')
    )
])

if __name__ == '__main__':
   app.run(debug=True)


<IPython.core.display.Javascript object>